In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F 

from torchtext import datasets
from torchtext.data import Field
from torchtext import data

In [0]:
#!mkdir data
#!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/labels.txt
#!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/reviews.txt
#!mv *.txt data/
#import numpy as np
#with open('data/reviews.txt', 'r') as f:
#     reviews = f.readlines()
#with open('data/labels.txt', 'r') as f:
#     labels = f.readlines()

In [0]:
##import pandas as pd

##df =pd.concat([pd.DataFrame(reviews,columns=['reviews']),pd.DataFrame(labels,columns=['labels'])],axis=1)
#del reviews,labels

#############
#df['labels'] = np.where(df['labels']=='positive\n', 1,0)
#df.head()

In [4]:
import numpy as np
text = ['good movies'
,'awfull movie'
,'very good movie to watch'
,'very good movie to watch on weekends'
,'awfull movie wasted my money'
,'awfull movie wasted my money and time probably do something better'
,'great movie to watch anytime really good movie enjoy a lot'
,'awfull awfull awfull awfull movies wasted wasted wasted money money money'
,'bad movie awfully bad'
,'great movie but awfull acting really enjoy but wasted money']

label = [1,0,1,1,0,0,1,0,0,1]
print('length of text',len(text))
print('length of label',len(label))


length of text 10
length of label 10


In [5]:
from string import punctuation
all_text = '\n'.join([c for c in text if c not in punctuation])
print(all_text)

good movies
awfull movie
very good movie to watch
very good movie to watch on weekends
awfull movie wasted my money
awfull movie wasted my money and time probably do something better
great movie to watch anytime really good movie enjoy a lot
awfull awfull awfull awfull movies wasted wasted wasted money money money
bad movie awfully bad
great movie but awfull acting really enjoy but wasted money


In [6]:
all_text.split('\n')

['good movies',
 'awfull movie',
 'very good movie to watch',
 'very good movie to watch on weekends',
 'awfull movie wasted my money',
 'awfull movie wasted my money and time probably do something better',
 'great movie to watch anytime really good movie enjoy a lot',
 'awfull awfull awfull awfull movies wasted wasted wasted money money money',
 'bad movie awfully bad',
 'great movie but awfull acting really enjoy but wasted money']

In [7]:
from collections import Counter

all_text_1 = ' '.join(all_text.split('\n'))
words = all_text_1.split()
count_words = Counter(words)
total_words = len(words)
sorted_words = count_words.most_common(total_words)

vocab_size = len(sorted_words)
print('Vocab size',vocab_size)

Vocab size 28


In [0]:
char_to_int = {w:i + 1 for i, (w,c) in enumerate(sorted_words)}

In [0]:
### Creating Encoded data
text_encoded = list()
for review in all_text.split('\n'):
  i_x = [char_to_int[w] for w in review.split()]
  text_encoded.append(i_x)

In [10]:
text_encoded

[[5, 8],
 [2, 1],
 [9, 5, 1, 6, 7],
 [9, 5, 1, 6, 7, 16, 17],
 [2, 1, 3, 10, 4],
 [2, 1, 3, 10, 4, 18, 19, 20, 21, 22, 23],
 [11, 1, 6, 7, 24, 12, 5, 1, 13, 25, 26],
 [2, 2, 2, 2, 8, 3, 3, 3, 4, 4, 4],
 [14, 1, 27, 14],
 [11, 1, 15, 2, 28, 12, 13, 15, 3, 4]]

In [11]:
from keras.preprocessing.sequence import pad_sequences
text_encoded_padded = pad_sequences(text_encoded, padding='pre')
text_encoded_padded

Using TensorFlow backend.


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  1],
       [ 0,  0,  0,  0,  0,  0,  9,  5,  1,  6,  7],
       [ 0,  0,  0,  0,  9,  5,  1,  6,  7, 16, 17],
       [ 0,  0,  0,  0,  0,  0,  2,  1,  3, 10,  4],
       [ 2,  1,  3, 10,  4, 18, 19, 20, 21, 22, 23],
       [11,  1,  6,  7, 24, 12,  5,  1, 13, 25, 26],
       [ 2,  2,  2,  2,  8,  3,  3,  3,  4,  4,  4],
       [ 0,  0,  0,  0,  0,  0,  0, 14,  1, 27, 14],
       [ 0, 11,  1, 15,  2, 28, 12, 13, 15,  3,  4]], dtype=int32)

In [12]:
input_feature = Variable(torch.Tensor(text_encoded_padded).long())
label_tensor = Variable(torch.Tensor(label).float())
print('shape of Input',input_feature.shape)
print('shape of output',label_tensor.shape)


shape of Input torch.Size([10, 11])
shape of output torch.Size([10])


In [13]:
label_tensor

tensor([1., 0., 1., 1., 0., 0., 1., 0., 0., 1.])

In [0]:
### building RNN (many-to-one)

class RNN(nn.Module):
  '''
      1. nn.Embedding
      Input: batch_size x seq_length
      Output: batch-size x seq_length x embedding_dimension

      2. nn.LSTM
      Input: seq_length x batch_size x input_size (embedding_dimension in this case)
      Output: seq_length x batch_size x hidden_size
      last_hidden_state: batch_size, hidden_size
      last_cell_state: batch, hidden_size
      
      3. nn.Linear
      Input: batch_size x input_size (hidden_size of LSTM in this case or ??)
      Output: batch_size x output_size
 
  '''

  def __init__(self,n_vocab,n_embed,hidden_size,output_size):
    super().__init__()

    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(n_vocab+1,n_embed) ## n_vocab is unique words in dictionary ## n_embed is hyperparameter
     # input & output will has batch size as 1s dimension. e.g. (batch, time_step/seq_len, input_size)
     ## n_layers default is 1 if 2 which means stacking
    self.rnn = nn.RNN(n_embed, hidden_size, num_layers = 1, batch_first = True)
    self.fc =  nn.Linear(hidden_size,output_size)   

  def forward(self,x):
    '''
    We have to run all over once on whole input sequence (many-to-one)
    input-size = 1*10 we want output size =[1,1,2]   
    '''
    x = x # input batch_size * seq_length
    batch_size = x.size(0) 
    #print('Batch Size is',batch_size)
    x = self.embedding(x) # batch-size x seq_length x embedding_dimension
    x,hidden =self.rnn(x)   #batch-size x seq_length x hidden_size
    out = self.fc(hidden.squeeze(0))  ## x[:,-1,:]  should be equals to hidden.squeeze(0)
    out = torch.sigmoid(out)
    return out

In [0]:
## model
n_vocab = len(char_to_int) ## 28
n_embed = 100
hidden_size = 8
output_size =1
model = RNN(n_vocab,n_embed, hidden_size,output_size)


In [16]:
model
#model(input_features)

RNN(
  (embedding): Embedding(29, 100)
  (rnn): RNN(100, 8, batch_first=True)
  (fc): Linear(in_features=8, out_features=1, bias=True)
)

In [0]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [19]:
n_iter = 20
for epoch in range(n_iter):
  optimizer.zero_grad()
  prediction = model(input_feature).squeeze(1)
  loss = criterion(prediction, label_tensor)
  acc = binary_accuracy(prediction, label_tensor)
  loss.backward()
  optimizer.step()

  if epoch % 2 == 0:
    print('epoch',epoch,'loss',loss.item(),'accuracy',acc)




epoch 0 loss 0.6896332502365112 accuracy tensor(0.5000)
epoch 2 loss 0.6895902156829834 accuracy tensor(0.5000)
epoch 4 loss 0.6895471215248108 accuracy tensor(0.5000)
epoch 6 loss 0.6895040273666382 accuracy tensor(0.5000)
epoch 8 loss 0.6894609928131104 accuracy tensor(0.5000)
epoch 10 loss 0.6894180178642273 accuracy tensor(0.5000)
epoch 12 loss 0.6893749237060547 accuracy tensor(0.5000)
epoch 14 loss 0.6893319487571716 accuracy tensor(0.5000)
epoch 16 loss 0.689288854598999 accuracy tensor(0.5000)
epoch 18 loss 0.689245879650116 accuracy tensor(0.5000)
